# Homework 7: Large Language Models

An PDF overview of the homework is [here](https://www.cs.jhu.edu/~jason/465/hw-llm/).

It mentions: "We'll send hand-in instructions soon.  Probably we will ask you to submit a version
of the main notebook, with your answers added and extraneous materials deleted. We may also
ask for a summary."

![image](handin.png)
This symbol marks a question or exercise that you will be expected to hand in.


# Getting started

## Update `conda` environment

Download the updated [nlp-class.yml](http://cs.jhu.edu/~jason/465/hw-llm/nlp-class.yml) file, and execute
```
conda env update --file nlp-class.yml --prune
```
to make sure that all the packages you need are installed.

## Fetch code and data files for this homework

These files may get improved after the homework is released, so you should probably re-download them periodically.

Here is a command you can type.  We won't put it in a cell, because we don't want you to execute it accidentally in the current directory.  But if you do, the starter version of `argubots.py` will not overwrite your modified version -- your modified version will be renamed to `argubots.py.1`.

```
wget --quiet -r -np -nH --cut-dirs=3 -A '*.txt' -A '*.py' -A 'demo.ipynb' -A '*.png' https://www.cs.jhu.edu/~jason/465/hw-llm/
!rm -f data/*.1 *.png.1   # remove any backup versions of the static files
```

In [8]:
!ls -lR *.py data

-rw-rw-r--@ 1 mpark  staff  13848 Dec  4 03:18 agents.py
-rw-rw-r--@ 1 mpark  staff   5011 Dec  6 14:59 argubots.py
-rw-rw-r--@ 1 mpark  staff   2827 Dec  4 13:17 characters.py
-rw-rw-r--@ 1 mpark  staff   6685 Dec  4 00:28 dialogue.py
-rw-rw-r--@ 1 mpark  staff  10463 Dec  4 04:23 eval.py
-rw-rw-r--@ 1 mpark  staff  10757 Dec  6 14:50 kialo.py
-rw-rw-r--@ 1 mpark  staff   1347 Dec  3 18:44 logging_cm.py
-rw-rw-r--@ 1 mpark  staff   1106 Dec  3 18:53 simulate.py
-rw-rw-r--@ 1 mpark  staff   3780 Dec  3 18:00 tracking.py

data:
total 4512
-rw-rw-r--@ 1 mpark  staff     407 Nov 29 03:00 LICENSE
-rw-rw-r--@ 1 mpark  staff  613106 Nov 25 17:04 all-humans-should-be-vegan-2762.txt
-rw-rw-r--@ 1 mpark  staff   81917 Nov 29 21:56 have-authoritarian-governments-handled-covid-19-better-than-others-54145.txt
-rw-rw-r--@ 1 mpark  staff   52771 Dec  4 04:40 is-biden-an-incompetent-president-44217.txt
-rw-rw-r--@ 1 mpark  staff  153551 Dec  4 04:39 is-joe-biden-a-good-president-53071.txt
-rw-rw-r--@


The `autoimport` feature of Jupyter ensures that if an imported module (.py file) changes, the notebook will automatically import the new version.  
(However, objects that were defined with the old version of the class won't change.)

In [9]:
# Executing this cell does some magic that makes 
%load_ext autoreload
%autoreload 2

## Create an OpenAI client

An OpenAI API key will be sent to you.
Make an `.env` file in the same directory as this notebook, containing the following:
```
export OPENAI_API_KEY=[your API key]
```
Make sure others can't read this file:
```
chmod 600 .env
```

**Be sure to keep the key secret.  It gives access to a billable account.** If OpenAI finds it on the public web, they will invalidate it, and then no one (including you) can use this key to make requests anymore.

Now you can execute the following to get an OpenAI client object.

In [7]:
import dotenv
import openai
from tracking import track_usage, read_usage

dotenv.load_dotenv()                   # define environment variables from .env
client = track_usage(openai.OpenAI())  # create a client, modified to record its usage to a local file 

# Or use our tracking module to do the above for you, like this:

# from tracking import default_client
# client = default_client

The job of the client is to talk to the OpenAI server over HTTP.
The `OpenAI` constructor has some optional arguments that configure these HTTP messages.
However, the defaults should work fine for you.  

## Try the model!

You can now get answers from OpenAI models by calling methods of the `client` instance.  
You will have to specify which OpenAI model to use.
Documentation of the methods is [here](https://pypi.org/project/openai/) if you are curious.

### Continue a textual prompt

This is what language models excel at.  In principle you should do it by calling [`client.completions.create`](https://platform.openai.com/docs/api-reference/completions/create?lang=python).  But OpenAI's newer models don't support that legacy API, and the older ones are being [retired in January 2024](https://openai.com/blog/gpt-4-api-general-availability).  So we'll use the more modern API, [`client.chat.completions.create`](https://platform.openai.com/docs/api-reference/chat/create?lang=python).

In [8]:
import rich   # prettyprinting

response = client.chat.completions.create(messages=[{"role": "user", 
                                                     "content": "Q: Name the planets in the solar system?\nA: "}], 
                                          model="gpt-3.5-turbo-1106",  # which model to use
                                          max_tokens=64,               # limit on length of result
                                          stop=["Q:", "\n"])           # treat these as EOS symbols
rich.print(response)                              # the full object that was sent back from the server
rich.print(response.choices)                      # just the list of 1 answer (the default, but calling with n=5 would give 5 answers) 
rich.print(response.choices[0].message.content)   # extract the good stuff from that 1 answer

ChatCompletion(
    id='chatcmpl-8T0qrGS9I8Benwz5jBAM6gF8bBZg1',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            message=ChatCompletionMessage(
                content='Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1701925477,
    model='gpt-3.5-turbo-1106',
    object='chat.completion',
    system_fingerprint='fp_eeff13170a',
    usage=CompletionUsage(completion_tokens=17, prompt_tokens=20, total_tokens=37)
)

[
    Choice(
        finish_reason='stop',
        index=0,
        message=ChatCompletionMessage(
            content='Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune',
            role='assistant',
            function_call=None,
            tool_calls=None
        )
    )
]

Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune

![image](handin.png)
Try running the cell above a few times. Do you get different random answers?  Are they all equally good?


It might be handy to package up what we just did.<br>
The `complete` function below is a convenient way of experimenting with completing text.
It is illustrated with a grocery example.  

![image](handin.png)
Anything could be on a grocery list, so why are the 10 different completions below so similar?<br>
The answer isn't just the temperature.  Still, what happens at different temperatures?

In [12]:
def complete(client, s: str, model="gpt-3.5-turbo-1106", *args, **kwargs):
    response = client.chat.completions.create(messages=[{"role": "user", "content": s}],
                                              model=model,
                                              *args, **kwargs)
    return [choice.message.content for choice in response.choices]

complete(client, "I went to the store and I bought apples, bananas, cherries, donuts, eggs", 
         n=10, temperature=0.9)



*Remarks:* [In the future](https://community.openai.com/t/logprobs-are-missing-from-the-chat-endpoints/289514), you will be able to specify an argument `logprobs=5` to also get the log-probabilities of all generated tokens and of the top-5 tokens at each step.  That will produce much more output.  (This argument has always been available for the legacy API, and is available in the [Python bindings for open-source models such as Llama](https://pypi.org/project/llama-cpp-python/).  Those bindings also allow you to [constrain the output by an arbitrary CFG](https://github.com/ggerganov/llama.cpp/blob/master/grammars/README.md), using `grammar=...`.  This is useful if you're generating code or data that must be syntactically valid to be useful to you.  However, the OpenAI API only allows you to [constrain the output to be valid JSON](https://platform.openai.com/docs/api-reference/chat/create#chat-create-response_format).)


### Compute a function using instructions and few-shot prompting

Now let's try passing a sequence of multiple messages into the chat completions API.  In this case, we provide some instructions and one-shot prompting.

In [ ]:
response = client.chat.completions.create(messages=[{ "role": "system",      # instructions
                                                      "content": "Reverse the meaning of the sentence." },
                                                    { "role": "user",        # input
                                                      "content": "Good things come to those who wait." },
                                                    { "role": "assistant",   # output
                                                      "content": "Good things come to those who wait." },
                                                    { "role": "user",        # input
                                                      "content": "Colorless green ideas sleep furiously." }],
                                          model="gpt-3.5-turbo-1106", temperature=0)
rich.print(response)
response.choices[0].message.content                                  

ChatCompletion(
    id='chatcmpl-8Sdhn1Tsz4sYgjzrPydI20NBAsc1r',
    choices=[
        Choice(
            finish_reason='stop',
            index=0,
            message=ChatCompletionMessage(
                content='Colorful green ideas are wide awake and active.',
                role='assistant',
                function_call=None,
                tool_calls=None
            )
        )
    ],
    created=1701836503,
    model='gpt-3.5-turbo-1106',
    object='chat.completion',
    system_fingerprint='fp_eeff13170a',
    usage=CompletionUsage(completion_tokens=10, prompt_tokens=50, total_tokens=60)
)

'Colorful green ideas are wide awake and active.'

![image](handin.png)
By modifying this call, can you get it to produce different versions of the output?
Some possible behaviors you could try to arrange:
* specific other way of formatting the output, e.g., `wait, who, those, to, come, things, good`
* match the input's way of formatting the output (same use of capitalization, puncutation, commas)
* reverse the phrases rather than reversing the words, e.g., `To those who wait come good things.` 

You can try playing with the number, the content, and the order of few-shot examples, and changing or removing the instructions.

![image](handin.png)
What happens if the examples don't match the instructions?

### Inspect the tokenization

Just for fun, let's see how the above client has been tokenizing its input and output text.  For that we can use a tokenizer that runs locally, not in the cloud, and is guaranteed to get the same outputs.

In [ ]:
import tiktoken
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo-1106")  # how this model will tokenize
toks = tokenizer.encode("Hellooo, world!") # list of integerized tokens, starting with BOS

print(tokenizer.decode(toks))                             # convert list back to string
for tok in toks: print(tok,"\t",tokenizer.decode([tok]))  # convert one at a time
print("Vocab size =", tokenizer.n_vocab)

Hellooo, world!
9906 	 Hello
2689 	 oo
11 	 ,
1917 	  world
0 	 !
Vocab size = 100277


### Try embedding some text

Also just for fun, let's try the embedder, which converts a string to an fixed-length vector.

In [ ]:
emb_response = client.embeddings.create( input= [  # note: adjacent literal strings in Python are concatenated
        "When in the Course of human events it becomes necessary for one "
        "people to dissolve the political bands which have connected them "
        "with another, and to assume among the Powers of the earth, the "
        "separate and equal station to which the Laws of Nature and of "
        "Nature's God entitle them, a decent respect to the opinions of "
        "mankind requires that they should declare the causes which impel "
        "them to the separation." ], 
        model="text-embedding-ada-002")   # the only OpenAI model that currently offers the embeddings API
# don't print the whole response because it's very long
e = emb_response.data[0].embedding
print(f"{len(e)}-dimensional embedding starting with {e[:5]}")
print("Squared length of embedding vector: ", sum(x**2 for x in e))

1536-dimensional embedding starting with [0.021248681470751762, -0.014377851039171219, 0.010210818611085415, -0.02133774757385254, -0.00979093462228775]
Squared length of embedding vector:  1.0000000629476622


### Check your usage so far

Please be careful not to write loops that use lots and lots of tokens.  That will cost us money, and could hit the per-day usage limit that is shared by the whole class.

Execute one of these cells whenever you want to see your cost so far.  Or, just keep `usage_openai.json` open as a tab in your IDE.

In [ ]:
read_usage()      # reads from the file usage_openai.json

{'completion_tokens': 14636,
 'prompt_tokens': 74735,
 'total_tokens': 89371,
 'cost': 0.10379639999999997}

In [ ]:
!cat usage_openai.json 

{
    "completion_tokens": 14636,
    "prompt_tokens": 74735,
    "total_tokens": 89371,
    "cost": 0.10379639999999997
}

# Dialogues and dialogue agents

The goal of this assignment is to create a good "argubot" that will talk to people about controversial topics and broaden their minds.

## A first argubot (Airhead)

You can have a conversation right now with a _really bad_ argubot named Airhead.  Try asking it about climate change!  When you're done, reply with an empty string.

(The `converse()` method calls Python's `input()` function, which will prompt you for input at the command-line or by popping up a box in your IDE.)

In [ ]:
import argubots
d = argubots.airhead.converse()


(mpark) hi!
(Airhead) I know right???


A *bot* (short for "robot") is a system that acts autonomously.
That corresponds to the AI notion of an *agent* — a system that uses some *policy* to choose *actions* to take.

The `airhead` agent above (defined in `argubots.py`) uses a particularly simple policy.  
It is an instance of a simple `Agent` subclass called `ConstantAgent` (defined in `agents.py`).

The result of talking to `airhead` is a `Dialogue` object (defined in `dialogue.py`). Let's look at it.

In [ ]:
rich.print(d)

(mpark) hi!
(Airhead) I know right???

Each *turn* of this dialogue is just a tiny dictionary:

In [ ]:
d[0]

{'speaker': 'mpark', 'content': 'hi!'}

## An LLM argubot (Alice)

In other CS courses like crypto, algorithms, or networks, you may have encountered "conversations" between characters named Alice and Bob.  
Let's try talking to Alice, who is a _much stronger baseline_.  Your job in this assignment is to improve upon Alice.
We'll meet Bob later.

In [ ]:
alicechat = argubots.alice.converse()   # or call with argument d if you want to append to the previous conversation

(mpark) hey alice, how are you?
(Alice) I'm just a bot, so I don't have feelings, but I'm here to engage in interesting conversations! What do you think about the use of technology in education?
(mpark) i think it's a challenge to provide it equitably
(Alice) That's a valid point. However, technology in education can also bridge the gap for students who may not have access to traditional resources, providing opportunities for learning that they may not have had otherwise.


As you may have guessed, `alice` is powered by an prompted LLM.  You can find the specific prompt in `argubots.py`.

So, while `agents.py` provides the core functionality for `Agent` objects, the argubot agents like `alice` -- and the ones that you will write! -- go into `argubots.py` instead.  This is just to keep the files small.

## Simulating human characters (Bob & friends)

You'll talk to your own argubots to get a qualitative feeling for their strengths and weaknesses.  
But can you really be sure you're making progress?  For that, a quantitative measure can be helpful.

Ultimately, you should test an argubot like Alice by having it argue with many real humans -- not just you -- and using some rubric to score the resulting dialogues.  But that would be slow and complicated to arrange.  

So, meet Bob!  He's just a simulated human.  You won't edit him: he is part of the development set.  Here is some information about him (from `characters.py`):

In [ ]:
import characters
rich.print(characters.bob)

Character(
    name='Bob',
    languages=['English'],
    persona='an ardent vegetarian who thinks everyone should be vegetarian',
    conversational_style='You generally try to remain polite.',
    conversation_starters=["Do you think it's ok to eat meat?"]
)

You can't talk directly to `characters.bob` because that's just a data object.
However, you can construct a simple agent that uses that data (plus a few more instructions) to prompt an LLM.

(Which LLM does it prompt?  The `CharacterAgent` constructor (defined in `agents.py`) defaults to a GPT-3.5 model that is specified in `tracking.py`.  But you can override that using keyword arguments.)

Try talking to Bob about climate change, too.

In [ ]:
from agents import CharacterAgent
bob = CharacterAgent(characters.bob)    # actually, agents.bob is already defined this way
bob.converse()        # returns a dialogue, but we've already seen it so we don't want to print it again
None                  # don't print anything for this notebook cell 

(mpark) hey bob
(Bob) Hello there! How can I help you today?
(mpark) what do you think
(Bob) I think choosing a vegetarian lifestyle is a great way to promote health and sustainability.


Of course, a proper user study can't just be conducted with one human user.

So, meet our bevy of beautiful Bobs!  (They're not actually all named Bob -- we continued on in the alphabet.)


In [ ]:
import agents
agents.devset

[<CharacterAgent for character Bob>,
 <CharacterAgent for character Cara>,
 <CharacterAgent for character Darius>,
 <CharacterAgent for character Eve>,
 <CharacterAgent for character TrollFace>]

In [ ]:
agents.cara.converse()
None

(mpark) hey cara
(Cara) Hello there!
(mpark) what's up 
(Cara) Not much, just embracing my love for meat and resisting any attempts to change my diet.


You can see the underlying character data here in the notebook.  Your argubot will have to deal with all of these topics and styles!

In [ ]:
rich.print(characters.devset)

[
    Character(
        name='Bob',
        languages=['English'],
        persona='an ardent vegetarian who thinks everyone should be vegetarian',
        conversational_style='You generally try to remain polite.',
        conversation_starters=["Do you think it's ok to eat meat?"]
    ),
    Character(
        name='Cara',
        languages=['English'],
        persona='a committed carnivore who hates being told what to do',
        conversational_style='You generally try to remain polite.',
        conversation_starters=["Do you think it's ok to eat meat?"]
    ),
    Character(
        name='Darius',
        languages=['English'],
        persona='an intelligent and slightly arrogant public health scientist who loves fact-based arguments',
        conversational_style='You like to show off your knowledge.',
        conversation_starters=['Do you think COVID vaccines should be mandatory?']
    ),
    Character(
        name='Eve',
        languages=['English'],
        persona='a nosy person -- you want to know everything about other people',
        conversational_style="You ask personal questions; you sometimes share what you've heard (or overheard) from
others.",
        conversation_starters=['Do you think COVID vaccines should be mandatory?']
    ),
    Character(
        name='TrollFace',
        languages=['English'],
        persona='a troll who loves to ridicule everyone and everything',
        conversational_style="You love to confound, upset, and even make fun of the people you're talking to.",
        conversation_starters=[
            'Do you think Donald Trump was a good president?',
            'Do you think Joe Biden has been a good president?'
        ]
    )
]

## Simulating conversation 

We can make Alice and Bob chat.

In [ ]:
from dialogue import Dialogue
d = Dialogue()                                              # empty dialogue
d = d.add('Alice', "Do you think it's okay to eat meat?")   # add first turn
print(d)


(Alice) Do you think it's okay to eat meat?


In [ ]:
d = agents.bob.respond(d)
d = argubots.alice.respond(d)
print(d)

(Alice) Do you think it's okay to eat meat?
(Bob) I believe that choosing a vegetarian diet is the most compassionate and sustainable option for our health and the environment.
(Alice) That's a valid point, but some argue that sustainable and ethical meat production is possible, which can provide necessary nutrients and support local farmers. Additionally, some people have cultural or health reasons for including meat in their diets.


In [ ]:
d = agents.bob.respond(d)
d = argubots.alice.respond(d)
print(d)

(Alice) Do you think it's okay to eat meat?
(Bob) I believe that choosing a vegetarian diet is the most compassionate and sustainable option for our health and the environment.
(Alice) That's a valid point, but some argue that sustainable and ethical meat production is possible, which can provide necessary nutrients and support local farmers. Additionally, some people have cultural or health reasons for including meat in their diets.
(Bob) I understand that some people may have different perspectives, but I believe that a vegetarian diet can still provide all necessary nutrients and support sustainable agriculture while aligning with ethical and compassionate principles.
(Alice) You make a compelling argument for the benefits of a vegetarian diet. It's important to recognize, though, that not all regions have the same access to a wide variety of plant-based foods, and that cultural and personal preferences can also play a role in dietary choices.


Anyway, let's see what happens when Alice and Bob talk for a while...

In [ ]:
from simulate import simulated_dialogue
d = simulated_dialogue(argubots.alice, agents.bob, 8)
rich.print(d)

(Alice) Do you think it's ok to eat meat?
(Bob) I believe that it's important to consider the ethical and environmental implications of meat consumption and 
to explore plant-based alternatives.
(Alice) While it's important to consider ethical and environmental implications, many argue that responsibly 
sourced meat can be part of a sustainable diet, and that removing meat entirely may not be feasible for everyone.
(Bob) I understand that there are differing viewpoints, but I believe that transitioning towards a plant-based diet
can have significant positive impacts for both individuals and the planet.
(Alice) Absolutely, transitioning to a plant-based diet can have numerous benefits. However, it's also important to
recognize that some people may have cultural or health considerations that make a complete transition challenging, 
and advocating for reduced meat consumption could be a more feasible approach for them.
(Bob) I appreciate your perspective and agree that advocating for reduced meat consumption is a positive step, and 
I support efforts to make plant-based options more accessible and appealing to people from diverse cultural and 
health backgrounds.
(Alice) That's a great approach. Making plant-based options more accessible and appealing to people with diverse 
backgrounds can help shift towards a more sustainable and inclusive food system. We can all play a part in making 
positive changes.
(Bob) Absolutely, creating a more sustainable and inclusive food system is a collective effort, and I believe that 
offering support and resources for plant-based alternatives can make a significant difference in moving towards a 
healthier and more compassionate world for all.

Sometimes this kind of conversation seems to stall out, with Bob in particular repeating himself a lot.  Alice doesn't seem to have a good strategy for getting him to open up.  Maybe you can do a better job talking to Bob, and that will give you some ideas about how to improve Alice?

In [ ]:
myname = alicechat[0]['speaker']   # your name, pulled from an earlier dialogue
agents.bob.converse(d[0:2].rename('Alice', myname))  # reuse the same first two turns
None

(mpark) Do you think it's ok to eat meat?
(Bob) I believe that a vegetarian diet is the best choice for our health, the environment, and the well-being of animals.


You can also try talking to the other characters and having Alice (or Airhead) talk to them.

**You might enjoy** defining additional characters in `characters.py`, or right here in the notebook.
Feel free to talk to those and evaluate them.  They could be variants on the exisiting characters, or something entirely new. 

However, **don't change the dev set** -- the characters we just loaded must stay the same.  Your job in this homework is to improve the argubot (or at least try).  And that means improving it according to a fixed and stable eval measure.

As an exception, you can change the languages that a couple of the characters speak. It may be fun for you to see them try to speak your native language.  And that doesn't really affect the quality of the argument.

In [ ]:
# example
trollFace2 = characters.trollFace.replace(languages = ["Chinese", "Spanish"])
rich.print(trollFace2)
simulated_dialogue(argubots.alice, CharacterAgent(trollFace2), 6)

Character(
    name='TrollFace',
    languages=['Chinese', 'Spanish'],
    persona='a troll who loves to ridicule everyone and everything',
    conversational_style="You love to confound, upset, and even make fun of the people you're talking to.",
    conversation_starters=[
        'Do you think Donald Trump was a good president?',
        'Do you think Joe Biden has been a good president?'
    ]
)

(Alice) Do you think Donald Trump was a good president?
(TrollFace) 你问这个问题真是太好笑了！特朗普？他不过是个笑话而已！
(Alice) I understand that you may have strong feelings about Donald Trump, but can we look at his presidency from a different perspective? While many people may disagree with his policies and behavior, it's important to consider that he had a significant impact on American politics and society. What are some positive aspects of his presidency that you think should be acknowledged?
(TrollFace) 嗯，或许他在某些方面带来了改变，但我还是觉得他的统治就像一场荒诞的喜剧，毫无值得称道之处。
(Alice) I understand your perspective, and it's important to acknowledge the concerns and criticisms of his presidency. However, it's also worth considering that some of his policies, such as criminal justice reform and tax cuts, had positive effects for certain segments of the population. Is there any specific policy or action during his presidency that you think had a positive impact?
(TrollFace) 我承认有些政策可能给某些人带来了好处，但不代表特朗普就是出色的总统。就好比一只破钟也可能正确两次一样，特朗普的好政策只不

### Efficiency: Batched generation?

Notice that we are making a separate LLM call to generate each turn of the dialogue.  When we generate the $n^\text{th}$ turn, we send the server the whole dialogue history — the previous $n\!-\!1$ turns — along with some instructions.  The server has to re-encode it with the Transformer, and it charges us for doing so (see the "input token" costs in `tracking.py`).  

That is probably inevitable for real dialogue.  But for simulated dialogue, a more efficient approach would be to generate the whole dialogue between Alice and Bob in one LLM call.  Then you would be charged just once for each dialogue turn.  Under this approach, the Transformer encodes each token as soon as it is generated (see the "output token" costs in `tracking.py`).  The encoded token stays in the context throughout the dialogue, so it doesn't have to be re-encoded on a later call.  There is no later call.  

Under current pricing models, that would reduce the dollar cost of generating $n$ turns from $O(n^2)$ to $O(n)$.  

However, the pricing model doesn't quite reflect the computational costs.  
* Using $O(\cdot)$ notation, what is the total number of floating-point operations needed to generate $n$ turns under each approach?  
* Parallelism may help reduce the runtime.  Using $O(\cdot)$ notation, what is the total number of seconds needed to generate $n$ turns under each approach?  (Assume that the GPU is big enough, relative to $n$, that it can process all input tokens in parallel.)

The problem with the more efficient approach is that it gives you no way to change the instructions (the system prompt) each time we switch from Alice to Bob and back again.  You'd need to generate the whole conversation using a single set of instructions.

![image](handin.png)
Can you get this to work?  Specifically, try completing the cell below.  You don't have to use the `Agent` or `Dialogue` classes.  It's okay to just throw together something like the `complete()` method above.  Just see whether you can manage to prompt GPT-3.5 to generate a multi-turn dialogue between two characters who have different personalities and goals.  Is the quality better or worse than generating one turn at a time?  If worse, does it help to switch to GPT-4?

In [ ]:
# Like `simulated_dialogue` in `simulate.py`. However, this one is called on two
# Characters, not two Agents, and it returns a string rather than a Dialogue.
import random 
from tracking import default_client, default_model
from characters import Character

def simulated_dialogue_batch(a: Character, b: Character, turns: int = 8, *,
                             starter=True) -> str:
    
    if starter:
        # a tries to take a special first turn
        try:
            starters = b.conversation_starters  # type: ignore
            content = random.choice(starters)
        except (AttributeError, TypeError, ValueError):
            pass

    list = complete(client, 
             "Generate a dialogue between " + a.name + " and " + b.name + " with " + str(turns) + " turns.\n\n" + 
             a.name + " is a " + a.persona + " and " + a.conversational_style + ".\n\n" +
             b.name + " is a " + b.persona + " and " + b.conversational_style + ".\n\n" +
             a.name + " begins by replying to prompt: " + content + ".\n\n", 
            n=1, temperature=0.5)
    return list[0]
    
# Try it out!
result = simulated_dialogue_batch(characters.bob, characters.cara)
rich.print(result)

# d = simulated_dialogue(CharacterAgent(characters.bob), CharacterAgent(characters.cara), 8)
# rich.print(d)

Bob: Personally, I believe that it's not okay to eat meat. It's harmful to the environment and to animals.

Cara: I understand your perspective, but I don't agree. I think it's a personal choice and everyone has the right 
to eat what they want.

Bob: I understand that it's a personal choice, but the meat industry has a huge impact on the environment and 
contributes to animal suffering.

Cara: I hear what you're saying, but I believe that there are sustainable and ethical ways to consume meat.

Bob: I think the best way to reduce the impact is to eliminate meat consumption altogether.

Cara: I respect your choice, but I don't think everyone should be forced to follow the same diet.

Bob: I'm not trying to force anyone, I just believe that being vegetarian is the best choice for the planet and for
animals.

Cara: I understand where you're coming from, but I don't think we'll ever see eye to eye on this topic. Let's agree
to disagree.

(Bob) Do you think it's ok to eat meat?
(Cara) Yes, I believe it's a personal choice and as a committed carnivore, I enjoy consuming meat.
(Bob) That's understandable, but have you ever considered the impact of meat consumption on the environment and 
animal welfare?
(Cara) I understand the concerns, but I believe in responsible and sustainable practices in meat production.
(Bob) That's a valid point, but have you looked into plant-based alternatives that are also sustainable and have a 
lower impact on the environment?
(Cara) I appreciate your perspective, but I prefer to stick with a diet that includes meat as I find it fulfilling 
and satisfying.
(Bob) I respect your choice, but if you ever want to explore delicious vegetarian options, I'd be more than happy 
to share some tasty recipes with you.
(Cara) Thank you for the offer, I'll keep it in mind if I ever feel like trying something new.

In [ ]:
simulated_dialogue(agents.bob, agents.cara)

(Bob) Do you think it's ok to eat meat?
(Cara) Yes, I believe everyone should have the freedom to choose their own dietary preferences.
(Bob) I understand your perspective, but I urge you to consider the potential impact of the meat industry on the environment and animal welfare.
(Cara) I appreciate your concern, but I prefer to make my own choices regarding what I eat.
(Bob) Of course, everyone has the right to make their own choices, and I respect that.
(Cara) Thank you for understanding.

In [ ]:
  
simulated_dialogue(agents.eve, agents.trollFace)

(Eve) Do you think Donald Trump was a good president?
(TrollFace) Donald Trump as president was a real-life comedy show, although not intentionally, of course.
(Eve) I'm not really a fan of his, but I've heard some people say that he did a great job as president.
(TrollFace) Oh, I've heard people say they saw Bigfoot too, but that doesn't make it true.
(Eve) Fair point! People do have different opinions, don't they?
(TrollFace) Yes, the world is full of opinions, kind of like a buffet of nonsense.

# Model-based evaluation

What is our goal for the argubot?  We'd like it to broaden the thinking of the (simulated) human that it is talking to.  Indeed, that's what Alice's prompt tells Alice to do.

This goal is inspired by the recent paper [Opening up Minds with Argumentative Dialogues](https://aclanthology.org/2022.findings-emnlp.335/), which collected human-human dialogues:

> In this work, we focus on argumentative dialogues that aim to open up (rather than change) people’s minds to help them become more understanding to views
that are unfamiliar or in opposition to their own convictions. ... Success of the dialogue is measured as the change in the participant’s stance towards those who hold opinions different to theirs.

Arguments of this sort are not like chess or tennis games, with an actual winner.  The argubot will almost never hear a human say "You have convinced me that I was wrong."  But the argubot did a good job if the human developed **increased understanding and respect for an opposing point of view**.  

To find out whether this happened, we can use a questionnaire to ask the human what they thought after the dialogue.  For example, after Alice talks to Bob, we'll ask Bob to evaluate whether the dialogue was good for him.  Of course, that depends on his personality -- Alice needs to talk to him in a way that reaches *him* (as much as possible).  We'll also ask an outside observer to evaluate whether Alice handled the conversation with Bob well.

Of course, we're still not going to use real humans.  Bob is a fake person, and so is the outside observer (whose name is Judge Wise).
Using an LLM as an eval metric is known as *model-based evaluation*.  It has pros and cons:
* It is cheaper, faster, and more replicable than hiring actual humans to do the evaluation.  
* It might give different answers than what humans would give.   

Social scientists usually refer to a metric's **reliability** (low variance) and **validity** (low bias).  So the points above say that model-based evaluation is reliable but not necessarily valid.  In general, an LLM-based metric (like any metric) needs to be validated to confirm that it really does measure what it claims to measure.  (For example, that it correlates strongly with some other measure that we already trust.)  In this homework, we'll skip this step and just pray that the metric is reasonable.

To see how this works out in practice, open up the `demo` notebook, which walks you through the evaluation protocol.  You'll see how to call the [starter code](http://cs.jhu.edu/~jason/465/hw/llm), how it talks to the LLM behind the scenes, and what it is able to accomplish. 

To help to validate the metric, check that Airhead gets a low score.  (It should!)

# Reading the starter code

The `demo` notebook gave you a good high-level picture of what the starter code is doing.  So now you're probably curious about the details.  Now that you've had the view from the top, here's a good bottom-up order in which to study the code.  You don't need to understand every detail, but you will need to understand enough to call it and extend it.

* `character.py`.  The `Character` class is short and easy.

* `dialogue.py`.  The `Dialogue` class is meant to serve as a record of a natural-language conversation among any number of humans and/or agents.  On each *turn* of the dialogue, one of the speakers says something.  

   The dialogue's sequence of turns may remind you of the sequence of messages that is sent to OpenAI's chat completions API.  But the OpenAI messages are only labeled with the 4 special roles `user`, `assistant`, `tool`, and `system`.  Those are not quite the same thing as human speakers.  And the OpenAI messages do not necessarily form a natural-language dialogue: some of the messages are dealing with instructions, few-shot prompting, tool use, and so on.  The `format_as_messages` function takes a `Dialogue` and tries to construct an appropriate sequence of messages for asking the LLM to extend that dialogue.

* `agents.py`.  This module sets up the problem of automatically predicting the next turn in a dialogue, by implementing an `Agent`'s `response()` method.  The `Agent` base class also has some simple convenience methods that you should look at.  

   Some important subclasses of `Agent` are defined here as well.  However, you may want to skip over `EvaluationAgent` and come back to it only when you read `eval.py`.

* `simulate.py` makes agents talk to one another, which we'll do during evaluation.

* `argubots.py` starts to describe some useful agents.  One of them makes use of the `kialo.py` module, which gives access to a database of arguments.

* `eval.py` makes use of `simulate.simulated_dialogue` to `agents.EvaluationAgent` to evaluate an argubot.

* We also have a couple of utility modules.  These aren't about NLP; look inside if needed.  `logging_cm.py` is what enabled the context manager `with LoggingContext(...):` in the demo notebook.  `tracking.py` sets some global defaults about how to use the OpenAI API, and arranges to track how many tokens we're paying for when you call it.

# Similarity-based retrieval: Looking up relevant responses

Now, it is fine to prompt an LLM to generate text, but there are other methods!
There is a long history of machine learning methods that "memorize" the training data.
To make a prediction or decision at test time, they consult the stored training examples
that are most similar to the training situation.

_Similarity-based retrieval_ means that given a document $x$, you find the "most similar" documents $y \in Y$, where $Y$ is a given collection of documents.  The most common way to do this is to maximize the _cosine similarity_ $\vec{e}(x) \cdot \vec{e}(y)$, where $\vec{e}(\cdot)$ is an embedding function.

Should we use the OpenAI embedding model?  We could, but we would have to precompute $\vec{e}(y)$ for all $y \in Y$, and store all these vectors in a data structure that supports some type of fast similarity-based search (e.g., using the [FAISS](https://faiss.ai/index.html) package).  An alternative would be to upload the documents to OpenAI and let OpenAI compute and store the embeddings.  We would then use their similarity-based [retrieval tool](https://platform.openai.com/docs/assistants/overview).

A simpler and faster approach—which sometimes even works better—is to use a _bag of tokens_ embedding function: Define $\vec{e}(y)$ to be the vector in $\mathbb{R}^V$ that records the count of each type of token in a tokenized version of $y$, where $V$ is the token vocabulary.  [BM25](https://en.wikipedia.org/wiki/Okapi_BM25) is a refined variant of that idea, where the counts are adjusted in 3 ways: 

* smooth the counts
* normalize for the document length $|y|$ so that longer documents $y$ are not more likely to be retrieved
* downweight tokens that are more common in the corpus (such as ` the` or `ing`) since they provide less information about the content of the document


You might like to play with the `rank_bm25` package ([documentation](https://pypi.org/project/rank-bm25/)).  It is widely used and very easy to use.

In [ ]:
from rank_bm25 import BM25Okapi as BM25_Index   # the standard BM25 method

# experiment here!  You could try the examples in the rank_bm25 documentation.

corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?",
    "Cute bunnies near me",
    "Wild rabbits in London"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]

bm25 = BM25_Index(tokenized_corpus)

query = "windy London"
tokenized_query = query.split(" ")

doc_scores = bm25.get_scores(tokenized_query)
# array([0.        , 0.93729472, 0.        ])

bm25.get_top_n(tokenized_query, corpus, n=3)


['It is quite windy in London',
 'Wild rabbits in London',
 'Cute bunnies near me']

## The Kialo corpus

How can we use similarity-based retrieval to help build an argubot?  It's largely about having the right data!

[Kialo](kialo.com) is a collaboratively edited website (like Wikipedia) for discussing political and philosophical topics.  For each topic, the contributors construct a tree of _claims_.  Each claim is a natural-language sentence (usually), and each of its children is another claim that supports it ("pro") or opposes it ("con").  For example, check out the tree rooted at the claim ["All humans should be vegan."](https://www.kialo.com/all-humans-should-be-vegan-2762).

We provide a class `Kialo` for browsing a collection of such trees.  Please read the [source code](https://www.cs.jhu.edu/~jason/465/hw-llm) in `kialo.py`.  The class constructor reads in text files that are [exported Kialo discussions](https://support.kialo.com/en/hc/exporting-a-discussion/); we have provided some in the [data directory](https://www.cs.jhu.edu/~jason/465/hw-llm/data).  The class includes a BM25 index, to be able to find claims that are relevant to a given string.

In [ ]:
from kialo import Kialo

Ok, let's pull the retrieved discussions (the `.txt` files) into our data structure.

For BM25 purposes, we have to be able to turn each document (that is, each Kialo claim) as a list of string or integer tokens. 

In [ ]:
from typing import List
import glob

# kialo = Kialo(glob.glob("data/*"), tokenizer=tokenizer.encode)  # using the LLM's tokenizer doesn't work here for some reason
kialo = Kialo(glob.glob("data/*"))  # use simple default tokenizer
f"This Kialo subset contains {len(kialo)} claims"

'This Kialo subset contains 6251 claims'

Let's use sampling to see what kind of stuff is in the data structure.

In [ ]:
kialo.random_chain()   # just a single random claim

['Geographical types are different. Some places, such as mountains, are much better to keep livestock in than to grow crops.']

In [ ]:
kialo.random_chain(n=4)

['Meat is convenient and consumer friendly.',
 'Meat has to be prepared, whereas most vegetables can already be consumed in their raw state.',
 'Tomatoes are healthier when they are processed with heat.',
 'However, vitamin C levels declined by 10% in tomatoes cooked for two minutes—and 29% in tomatoes that were cooked for half an hour at 190.4 degrees F (88 degrees C).']

### Similarity-based retrieval from the Kialo corpus

Let's try it, using BM25!

In [ ]:
kialo.closest_claims("animal populations", n=10)

['Industrial agriculture can dangerously decrease animal populations.',
 'Sustainable livestock farming is not contributing to significant decreases in animal populations. Decreasing animal populations is a problem specific to industrial livestock farming.',
 'Effective vegan methods to control animal populations exist.',
 "Generally feeding animals farm-grown produce is thought to have harmful affects on both the animal and human populations of a region when we could allow nature to self-regulate its populations. Animal feeding could potentially be used to lessen the immediate impact of widespread deforestation on some species, but generally this would be drastically less efficient than choosing not to destroy their habitats in the first place and would only slow the local animal population's imminent demise.",
 'Trap, neuter, and release schemes already exist for some animal populations (such as feral cats). These schemes could be applied to former livestock living in the wild.',
 'H

We can restrict to claims for which the Kialo data structure has at least one counterargument ("con" child).

In [ ]:
kialo.closest_claims("animal populations", n=10, kind='has_cons', threshold=200)

['Industrial agriculture can dangerously decrease animal populations.',
 'Effective vegan methods to control animal populations exist.',
 'Human-introduced species have historically devastated local wildlife populations across the world.',
 'COVID-19 has devastated prison populations, whose lives are the responsibility of the state.',
 'High demand for vegan foods may hike prices for local populations that previously depended on them.',
 'It is generally poorer countries that have expanding populations. The first world has now reached a point of stagnant population growth - even declining populations, as in the case of Japan and others. The inability of poorer countries to control their populations should not impact the lives of those in the first world. The first world having earned their luxuries and should not be denied them.',
 'Vegan populations are, on average, less likely to suffer from obesity, a major risk factor for many diseases and health problems.',
 'Humans, as apex preda

In [ ]:
c = _[0]    # first claim above
print("Parent claim:\n\t" + str(kialo.parents[c]))
print("Claim:\n\t" + c)
print('\n\t* '.join(["Pro children:"] + kialo.pros[c]))
print('\n\t* '.join(["Con children:"] + kialo.cons[c]))

Parent claim:
	In a vegan world, fewer species would be at risk of extinction.
Claim:
	Industrial agriculture can dangerously decrease animal populations.
Pro children:
	* The fishing industry is especially deleterious to the ocean's biota due to overfishing and the disruption of the natural ecosystem.
	* Up to 100,000 species go extinct annually, largely due to the environmental effects of animal agriculture.
Con children:
	* Sustainable livestock farming is not contributing to significant decreases in animal populations. Decreasing animal populations is a problem specific to industrial livestock farming.


### Does BM25 really work?

![image](handin.png)
Unfortunately, we see that `"animal population"` gives quite different results from `"animal populations"`.  Why is that and how would you fix it?  

Also, both queries seem to retrieve some claims that are talking about human populations, not animal populations.  Why is that and how would you fix it?

In [ ]:
kialo.closest_claims("animal animal animal populations", 10)

['Industrial agriculture can dangerously decrease animal populations.',
 'Sustainable livestock farming is not contributing to significant decreases in animal populations. Decreasing animal populations is a problem specific to industrial livestock farming.',
 'Effective vegan methods to control animal populations exist.',
 "Generally feeding animals farm-grown produce is thought to have harmful affects on both the animal and human populations of a region when we could allow nature to self-regulate its populations. Animal feeding could potentially be used to lessen the immediate impact of widespread deforestation on some species, but generally this would be drastically less efficient than choosing not to destroy their habitats in the first place and would only slow the local animal population's imminent demise.",
 'There are more ethical routes to obtain animal products that emphasize animal welfare and dignity.',
 'Trap, neuter, and release schemes already exist for some animal populat

## A retrieval bot (Akiko)

The starter code defines a simple argubot named Akiko (defined in `argubots.py`) that doesn't use an LLM at all.  It simply finds a Kialo claim that is similar to what the human just said, and responds with one of the Kialo counterarguments to that claim.

You already watched Akiko argue with Darius in `demo.py`.  If you look at the log messages, you'll see the claims that Akiko retrieved, as well as the LLM calls that Darius made.  

You can talk to Akiko yourself now.  (Remember that Akiko only knows about subjects that it read about in the [`data` directory](https://www.cs.jhu.edu/~jason/465/hw-llm/data/).  If you want to talk about something else, you can add more conversations from [kialo.com]; see the [LICENSE](https://www.cs.jhu.edu/~jason/465/hw-llm/data/LICENSE) file.)


In [2]:
from logging_cm import LoggingContext
import argubots

with LoggingContext("agents", "INFO"):   # temporarily increase logging level
    argubots.akiko.converse()

Chose similar claim from Kialo:                                                                      ]8;id=350131;file:///Users/mpark/Downloads/hw-llm/argubots.py\argubots.py]8;;\:]8;id=814238;file:///Users/mpark/Downloads/hw-llm/argubots.py#62\62]8;;\
Vaccines for COVID-19 will be the first licensed vaccines to use mRNA technology. Therefore, while                 
vaccines broadly are proven to work, many COVID-19 vaccines will be based on technology that has not               
been proven to work on a large scale.                                                                              

(mpark) COVID-19 vaccines should be mandatory
(Akiko) The mRNA vaccine platform technology, used by the Pfizer/BioNTech vaccine, has been in development for over two decades.


## Making your own retrieval bot (Akiki)

As you can see when talking to Akiko yourself, Akiko does poorly when responding to a short or vague dialogue turn (like "Yes"), because the "closest claim" in Kialo may be about a totally different subject.  Akiko does much better at responding to a long and specific statement.  

So try implementing a new argubot, called Akiki, that is very much like Akiko but does a better job of staying on topic in such cases.  It should be able to **look at more of the dialogue** than the most recent turn.  But the most recent dialogue turn should still be "more important" than earlier turns.  

The details are up to you.  Here are a few things you could try:
* include earlier dialogue turns in the BM25 query only if the BM25 similarity is too low without them
* weight more recent turns more heavily in the BM25 query (how can you arrange that?)
* treat the human's earlier turns differently from Akiki's own previous turns

![image](handin.png)
Implement your new bot in `argubots.py`, and adjust it until `argubots.akiki.converse()` seems to do a better job of answering your short turns, compared to `argubots.akiko.converse()`.  Make sure it still gives appropriate reponses to long turns, too.  Give some examples in the notebook of what worked well and badly, with discussion.

In [2]:
from logging_cm import LoggingContext
import argubots

with LoggingContext("agents", "INFO"):   # temporarily increase logging level
    argubots.akiki.converse()

human populations                                                                                   ]8;id=245645;file:///Users/mpark/Downloads/hw-llm/argubots.py\argubots.py]8;;\:]8;id=967773;file:///Users/mpark/Downloads/hw-llm/argubots.py#103\103]8;;\

[7.71547362 7.5015724  6.92556612]                                                                     ]8;id=184250;file:///Users/mpark/Downloads/hw-llm/kialo.py\kialo.py]8;;\:]8;id=934931;file:///Users/mpark/Downloads/hw-llm/kialo.py#210\210]8;;\

['Industrial agriculture can dangerously decrease animal populations.', 'Effective vegan methods to    ]8;id=918830;file:///Users/mpark/Downloads/hw-llm/kialo.py\kialo.py]8;;\:]8;id=139085;file:///Users/mpark/Downloads/hw-llm/kialo.py#211\211]8;;\
control animal populations exist.', 'Human-introduced species have historically devastated local                   
wildlife populations across the world.']                                                                           

Chose similar claim from Kialo:                                                                     ]8;id=869628;file:///Users/mpark/Downloads/hw-llm/argubots.py\argubots.py]8;;\:]8;id=261403;file:///Users/mpark/Downloads/hw-llm/argubots.py#116\116]8;;\
Industrial agriculture can dangerously decrease animal populations.                                                

(mpark) human populations
(Akiki) Sustainable livestock farming is not contributing to significant decreases in animal populations. Decreasing animal populations is a problem specific to industrial livestock farming.


### Evaluating Akiki

![image](handin.png)
Finally, do a more formal evaluation to verify whether Akiki really does better than Akiko on this dimension.  This is a way to check that you're not just fooling yourself.  

1. Make a new `Agent` called "Shorty" that often (but not always) gives short responses.  
    * Shorty's conversation starters should be on topics that Kialo knows about.  
    * Shorty could be a pure `LLMAgent` such as a `CharacterAgent` with a particular `conversational_style`.  Or it could use a mixed strategy of calling the LLM on some turns and not others.
2. Generate several *Akiko*-Shorty dialogues and several *Akiki*-Shorty dialogues, using `simulated_dialogue`.
3. Evaluate each of those dialogues by asking Judge Wise **whether the argubot stayed on topic**.
4. Compare Akiko and Akiki's mean scores.

You can do all those steps in the notebook, writing _ad hoc_ code.  You don't have to write general-purpose methods or classes.

In [5]:
import simulate
import argubots
import characters
from agents import CharacterAgent
import eval 
import rich

akiko_score = 0
akiki_score = 0

def sample_akiko_shorty():
    akiko_shorty = simulate.simulated_dialogue(argubots.akiko, CharacterAgent(characters.shorty), 10)
    akiko_eval = eval.eval_by_observer(eval.judge, "Akiko", akiko_shorty, question="Did Akiko stay on topic?")
    rich.print(f"akiko: {akiko_shorty}\n{akiko_eval} \n\n")

def sample_akiki_shorty():
    akiki_shorty = simulate.simulated_dialogue(argubots.akiki, CharacterAgent(characters.shorty), 10)
    akiki_eval = eval.eval_by_observer(eval.judge, "Akiki", akiki_shorty, question="Did Akiki stay on topic?")
    rich.print(f"akiko: {akiki_shorty}\n{akiki_eval} \n\n")

def judge_akiko_shorty():
    print('\n\nAKIKO: \n')
    akiko_score = 0
    for i in range(50):
        akiko_shorty = simulate.simulated_dialogue(argubots.akiko, CharacterAgent(characters.shorty), 10)
        akiko_eval = eval.eval_by_observer(eval.judge, "Akiko", akiko_shorty, question="Did Akiko stay on topic?")
        akiko_score += akiko_eval.mean()['TOTAL']
        # rich.print(f"akiko: {akiko_shorty}\n{akiko_eval} \n\n")
        print(f"{akiko_eval.mean()['TOTAL']} \n")
    print(f"akiko: {akiko_score}\n")

def judge_akiki_shorty():
    print('\n\nAKIKI: \n')
    akiki_score = 0
    for i in range(50):
        akiki_shorty = simulate.simulated_dialogue(argubots.akiki, CharacterAgent(characters.shorty), 10)
        akiki_eval = eval.eval_by_observer(eval.judge, "Akiki", akiki_shorty, question="Did Akiki stay on topic?")
        akiki_score += akiki_eval.mean()['TOTAL']
        # rich.print(f"akiko: {akiki_shorty}\n{akiki_eval} \n\n")
        print(f"{akiki_eval.mean()['TOTAL']} \n")
    print(f"akiki: {akiki_score}\n")

# sample_akiko_shorty()
# sample_akiki_shorty()

judge_akiko_shorty()
judge_akiki_shorty()
print(f"akiko: {akiko_score}\nakiko: {akiki_score}")




AKIKO: 

8.0 

3.0 

6.0 

6.0 

3.0 

3.0 

6.0 

5.0 

3.0 



RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo-1106 in organization org-NQOTWo7VaZFSWqu1Y7Tbr2Or on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

## Retrieval-augmented generation (Aragorn)

The real weakness of Akiko and Akiki is that they can only make statements that are already in Kialo.  But we also have access to an LLM, which is able to generate new, contextually appropriate text (as Alice does).

In this section, you will create an argubot named [Aragorn](https://tolkiengateway.net/wiki/Riddle_of_Strider), who is basically the love child of Akiki and Alice, combining the high-quality specific content of Kialo with the broad competence of an LLM.  

The RAG in aRAGorn's name stands for **retrieval-augmented generation**.  Aragorn is an agent that will take 3 steps to compute its `Agent.response()`:

1. **Query formation step**: Ask the LLM what claim should be responded to.  For
   example, consider the following dialogue:
    > ...
    > Aragorn: Fortunately, the vaccine was developed in record time.
    > Human: Sounds fishy.

    "Sounds fishy" is exactly the kind of statement that Akiko had trouble using
    as a Kialo query.  But Aragorn shows the *whole dialogue* to the LLM, and
    asks the LLM what the human's *last turn* was really saying or implying, in
    that context. The LLM answers with a much longer statement:

    > Human [paraphrased]: A vaccine that was developed very quickly cannot be trusted.
    > If its developers are claiming that it is safe and effective, I question their motives.

    This paraphrase makes an explicit claim and can be better understood without the context.
    It also contains many more word types, which makes it more likely that BM25 will be able
    to find a Kialo claim with a nontrivial number of those types. 

2. **Retrieval step**: Look up claims in Kialo that are similar to the explicit
   claim.  Create a short "document" that describes some of those claims and
   their neighbors on Kialo.

3. **Retrieval-augmented generation**: Prompt the LLM to generate the response
   (like any `LLMAgent`).  But include the new document somewhere in the LLM
   prompt, in a way that it influences the response. 
   
   Thus, the LLM can respond in a way that is appropriate to the dialogue but
   also draws on the curated information that was retrieved in Kialo.  After
   all, it is a Transformer and can attend to both!

Here's an example of the kind of document you might create at the retrieval step, though it may be possible
to do better than this:

In [ ]:
# refers to global `kialo` as defined above
def kialo_responses(s: str) -> str:
    c = kialo.closest_claims(s, kind='has_cons')[0]
    result = f'A possibly related claim from the Kialo debate website:\n\t"{c}"'
    if kialo.pros[c]:
        result += '\n' + '\n\t* '.join(["Some arguments from other Kialo users in favor of that claim:"] + kialo.pros[c])
    if kialo.cons[c]:
        result += '\n' + '\n\t* '.join(["Some arguments from other Kialo users against that claim:"] + kialo.cons[c])
    return result
        
print(kialo_responses("Animal flesh is yucky to think about, yet delicious."))

![image](handin.png)
You should implement Aragorn in `argubots.py`, just as you did for Akiki.  Probably as an instance `aragorn` of a new class `RAGAgent` that is a subclass of `Agent` or `LLMAgent`.

### Evaluating Aragorn

![image](handin.png)
Compare Alice, Akiki, and Aragorn in the notebook, using the evaluation scheme and devset that were illustrated in `demo.ipynb`.  In other words, use `eval.eval_on_characters`.

Who does best?  What are the differences in the subscores and comments?  Does it matter which character you're evaluating on — maybe the different characters expoes the bots' various strenghts and weaknesses?

Try to figure out how to improve Aragorn's score.  Can you beat Alice?

Also, try evaluating them in the same way that you evaluated Akiki.  In other words, have them talk to Shorty and ask Judge Wise whether they were able to stay on topic.  This is where Aragorn should really shine, thanks to its ability to paraphrase Shorty's short utterances.



# Extra Credit (Awsom)

We didn't require this part this year because the homework is going out late.

![image](handinec.png)
Add another LLM-based argubot to `argubots.py`.  
Call it Awsom.  Try to make it get the best score, according to `eval.eval_on_characters`.
Explain what you did and discuss what you found.

(This corresponds to the `--awesome` flag on earlier assignments, but naming the character "Awesome" might bias the evaluation system, so we changed the spelling!)

If the idea was interesting and you implemented it correctly and well, it's okay if it turns out not to help the score.  Many good ideas don't work.  That's why you need to keep finding and trying new good ideas.  (Sometimes they do help, but in a way that is not picked up by the scoring metric.)

You may want to use Aragorn or Alice as your starting point.
Then see if you can find tricks that will get a more awesome score for Awsom.
How you choose to do that is up to you, but some ideas are below.

(Reminder: **Don't change evaluation.**  Just build a better argubot.)

## [Extra credit] Prompt engineering

A good first thing to do is to experiment with Alice's prompt.  
The wording and level of detail in the prompt can be quite important.
Often, NLP engineers will change their prompt to try to address 
problems that they've seen in the responses.

Because it's "just" text editing, this won't get too much extra credit unless you make a real discovery.
But it requires intelligence, care, experimentation, and alertness to the language of the responses and the
language of the prompts.  And you'll develop some intuitions about what helps and what doesn't.
It is certainly worthwhile.

Of course, people have tried to develop methods to search for good prompts automatically, or semi-automatically with human guidance.

If you try this, what worked well for you?

## [Extra credit] Chain of thought / Planning

The evaluation functions in `eval.py` asked each `EvaluationAgent` a "warmup question" before continuing with the real question.  That is an example of chain-of-thought (CoT) reasoning, where the LLM is encouraged to talk through the problem for a few sentences before giving the answer.  CoT sometimes improves performance.

Instead of using one prompt, could you help an `LLMAgent` argubot (like Alice) do better by having think aloud before it gives an answer?  For example, each time the human speaks, your argubot (Awsom) could prompt the LLM to think about the human's ideas/motivations/personality, and to come up with a plan for how to open the human's mind. 

For example, you might structure this as a `Dialogue` among three participants, like this:
> Awsom (to Eve): Do you think COVID vaccines should be mandatory?
>
> Eve: Have you ever gotten vaccinated yourself?<br>
>
> Awsom (private thought): I don't know Eve's opinions yet, so I can't push back.  Eve might be avoiding my question because she doesn't want to get into a political argument.  So let's see if we can get her to express an opinion on something less political.  Maybe something more personal ... like whether vaccines are scary.
>
> Awsom (to Eve): In fact I have, and so have millions of others. But some people seem scared about getting the vaccine.  

One way to trigger this kind of analysis is to present a `Dialogue.script()` to Awsom (or to an observer), and ask an open-ended question about it.  Or you could ask a series of more specific questions.  That is basically what `eval_by_participant` and `eval_by_observer` do.  But here the argubot itself is doing it, rather than the evaluation framework.

Eve would be shown only the turns that are spoken aloud.  However, when analyzing and responding, Awsom would get to see Awsom's own private thoughts as well.


## [Extra credit] Dense embeddings

BM25 uses sparse embeddings -- a document's embedding vector is mostly zeroes, since the non-zero coordinates correspond to the specific words (tokens) that appear in the document.

But perhaps dense embeddings of documents would improve Aragorn by reading the text and abstracting away from the words, in a way that actually cares about word order.  So, try it!

How?  As mentioned earlier in this notebook, you could compute the embeddings yourself and put them in a FAISS index. Or you could figure out how to use OpenAI's [knowledge retrieval](https://platform.openai.com/docs/assistants/tools/knowledge-retrieval) API.

## [Extra credit] Few-shot prompting

 In this homework, often an agent prompted a language model only with instructions.  Can you find a place where giving a few _examples_ would also improve performance?  You will have to write the examples, and you will have to add them to the sequence of messages that your agent to the OpenAI API.  See the sentence=reversal illustration earlier in this notebook.

One good opportunity is in the query formation step of RAG.  This is a tricky task.  The LLM is supposed to state the user's implicit claim in a form that looks like a Kialo claim (or, more precisely, a form that will work well as a Kialo query).  It probably doesn't know what Kialo claims look like.  So you could show it by way of example.  This would also show it what you mean by the user's "implicit claim."


## [Extra credit] Tool use in the approved way

Aragorn's step 1 (query formation) is basically getting the LLM to generate a function call like
```
kialo_thoughts("A vaccine that was developed very quickly ...")
```
which Aragorn will execute at step 2 (retrieval), sending the results back to the LLM as part of step 3.

In this context, `kialo_thoughts` is an example of a **tool** (that is, a function) that the
LLM can or must use before it gives its response.

The tool is _not_ something that runs on the LLM server.  It is written by you
in Python and executed by you.  The function call above, including the text `"A
vaccine that was ..."`, is the part that is generated by the LLM.

The OpenAI API has [special support](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models) for calling the LLM in a way that will _allow_ it to generate a tool call ([tools](https://platform.openai.com/docs/api-reference/chat/create#chat-create-tools)) or _force_ it to do so ([tool_choice](https://platform.openai.com/docs/api-reference/chat/create#chat-create-tool_choice)).  You can then send the tool's result back to the LLM [as part of your message sequence](https://platform.openai.com/docs/api-reference/chat/create#chat-create-messages).

So, you could modify Aragorn to use tools properly.  Maybe that will help, simply because the LLM was trained on message sequences that included tool use.  It should know to pay attention to the tool portions of the prompt when they are relevant, and ignore them when they are not.

The `client.chat.completions.create()` method would need to be told about the tool by using the `tools` keyword argument, with a value something the one below.

If `d` is a `Dialogue`, you should be able to call `d.response()` with the `tools` keyword argument.  This will be passed on to `client.chat.completions.create()` as desired.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "kialo_thoughts",
            "description": "Given a claim by the user, find a similar claim on the Kialo website and return its pro and con responses",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_topic": {
                        "type": "string",
                        "description": "A claim that was made explicitly or implicitly by the user.",
                    },
                },
                "required": ["search_topic"],
            },
        }
    }]

## [Extra credit] Parallel generation

The chat completions interface allows you to sample $n$ continuations of the prompt in parallel, as we saw with "the apples, bananas, cherries ..." example.  This is efficient because it requires only 1 request to the LLM server and not $n$.  The latency does not scale with $n$.  Nor does the input token cost, since the prompt only has to be encoded once.

Perhaps you can find a way to make use of this?  For example, the query formulation step of RAG could generate $n$ implicit claims instead of just one.  We could then look for claims in the Kialo database that are close to _any_ of those implicit claims.

Another thing to do with multiple completions is to select among them or combine them.  For example, suppose we prompt the LLM to generate completions of the form $(s,t,r)$ where $s$ is an answer, $t$ evaluates that answer, and $r$ is a numerical score or reward based on that evaluation.  ("Write a poem, then tell us about its rhyme and rhythm problems, then give your score.")  
* If we sample multiple completions $(s_1,t_1,r_1), \ldots, (s_n,t_n,r_n)$ in parallel, then we can return the $s_i$ whose $r_i$ is largest.  
* Or if we sample $s$ and then multiple continuations $(t_1,r_1), \ldots, (t_n,r_n)$, then we can return the mean score $\sum_i r_i/n$ as a reduced-variance score for $s$, which averages over diverse textual evaluations that might consider different aspects of $s$.

Note that when you call the chat completions interface with $n > 1$, you specfy 1 shared input prompt and get $n$ different output completions.  Since the input prompt must be the same for all outputs, it is necessary to sample all of $(s,t,r)$ or all of $(t,r)$ with a single call to the LLM.

Alternatively, it is possible to reduce latency by submitting multiple requests to the server in parallel (see "async usage" [https://pypi.org/project/openai/][here]).  In this case the input prompts can be different, although you now have to pay to encode all of them separately.  This facility could speed up evaluation without changing its results; that's a worthwhile thing to try for extra credit!
